In [69]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data15070


In [70]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [71]:
!pip install seaborn

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [72]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple lightgbm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc, math

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
sns.set(rc={'figure.figsize':(11,8)})
sns.set(style="whitegrid")

In [18]:
%%time
metadata_df = pd.read_csv('/home/aistudio/data/data15070/building_m.csv')
train_df = pd.read_csv('/home/aistudio/data/data15070/train.csv', parse_dates=['timestamp'])
# 这个parse_dates的意思就是将csv中的时间字符串转换成日期格式
test_df = pd.read_csv('/home/aistudio/data/data15070/test.csv', parse_dates=['timestamp'])
weather_train_df = pd.read_csv('/home/aistudio/data/data15070/w_train.csv', parse_dates=['timestamp'])
weather_test_df = pd.read_csv('/home/aistudio/data/data15070/w_test.csv', parse_dates=['timestamp'])

In [19]:
train_df

In [20]:
weather_train_df

In [22]:
# 这个比赛是有时间戳的，但是这个时间戳还对不上，所以我们需要先对齐时间
weather = pd.concat([weather_train_df,weather_test_df],ignore_index=True)
weather_key = ['site_id', 'timestamp']

temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()
# subset这个的意思是去掉重复行就考虑weather_key这个
temp_skeleton

In [23]:
# calculate ranks of hourly temperatures within date/site_id chunks
temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')
# groupby就是拆分应用合并
# 先从原来的临时框架中拆分出site_id和timestamp的日部分，然后从这里面看温度，根据平均进行排序
# 就是对site_id和日期对每小时温度排序
temp_skeleton


In [24]:
# create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)
# unstack将聚合的两列一列值为行索引，另外一列为列索引

df_2d

In [25]:
# Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
# 用温度峰值-14，得到时间戳对齐间隔
site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
site_ids_offsets.index.name = 'site_id'

def timestamp_align(df):
    df['offset'] = df.site_id.map(site_ids_offsets)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned']
    return df

In [26]:
weather_train_df

In [27]:
weather_train_df = timestamp_align(weather_train_df)
weather_test_df = timestamp_align(weather_test_df)

In [28]:
del weather 
del df_2d
del temp_skeleton
del site_ids_offsets
weather_train_df

In [29]:
weather_train_df.isna().sum()

In [30]:
weather_train_df = weather_train_df.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))
# 先对site_id分组，然后对每个单独的进行apply操作
# 这个操作就是通过前后数据插值来填充NAN
# 这个both就是说处理前面和后面的nan
weather_test_df = weather_test_df.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))
weather_train_df

In [31]:
weather_train_df.isna().sum()
# 没想到还是有缺失值

In [32]:
train_df['meter_reading'] = np.log1p(train_df['meter_reading'])

In [33]:
## Function to reduce the memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [34]:
le = LabelEncoder()
# 这个是标准化标签，将标签值统一转换成rang（标签值个数-1）范围内的，反正感觉就是one-hot加归一化
metadata_df['primary_use'] = le.fit_transform(metadata_df['primary_use'])

In [35]:
metadata_df = reduce_mem_usage(metadata_df)
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)
weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)

In [36]:
%%time
full_train_df = train_df.merge(metadata_df, on='building_id', how='left')
full_train_df = full_train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')

In [37]:
full_train_df.shape

In [39]:
# Delete unnecessary dataframes to decrease memory usage
del train_df
del weather_train_df
gc.collect()
# 这个的作用是清理内存

In [43]:
gc.collect()

In [44]:
%%time
# full_test_df = test_df.merge(metadata_df, on='building_id', how='left')
# full_test_df = full_test_df.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')
full_test_df = test_df.merge(metadata_df, on='building_id', how='left')
full_test_df = full_test_df.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')
del metadata_df
del weather_test_df
del test_df
gc.collect()

In [ ]:
# 把数据处理完了开始EDA

In [45]:
ax = sns.barplot(pd.unique(full_train_df['primary_use']), full_train_df['primary_use'].value_counts())
# y轴是数据出现的频率
ax.set(xlabel='Primary Usage', ylabel='# of records', title='Primary Usage vs. # of records')
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
plt.show()

In [46]:
meter_types = {0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}
ax = sns.barplot(np.vectorize(meter_types.get)(pd.unique(full_train_df['meter'])), full_train_df['meter'].value_counts())
# 这个np.vectorize就是将函数向量化，当然在这好像就是加快速度用的
ax.set(xlabel='Meter Type', ylabel='# of records', title='Meter type vs. # of records')
plt.show()

In [48]:
ax = sns.barplot(np.vectorize(meter_types.get)(full_train_df.groupby(['meter'])['meter_reading'].mean().keys()), full_train_df.groupby(['meter'])['meter_reading'].mean())

ax.set(xlabel='Meter Type', ylabel='Meter reading', title='Meter type vs. Meter Reading')
plt.show()

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [49]:
fig, ax = plt.subplots(1,1,figsize=(14, 6))
ax.set(xlabel='Year Built', ylabel='# Of Buildings', title='Buildings built in each year')
full_train_df['year_built'].value_counts(dropna=False).sort_index().plot(ax=ax)
full_test_df['year_built'].value_counts(dropna=False).sort_index().plot(ax=ax)
ax.legend(['Train', 'Test']);

In [50]:
# 开始特征工程
pd.DataFrame(full_train_df.isna().sum().sort_values(ascending=False), columns=['NaN Count'])

In [53]:
# 然后把这些缺失值都给处理了
def mean_without_overflow_fast(col):
    col /= len(col)
    return col.mean() * len(col)
missing_values = (100-full_train_df.count() / len(full_train_df) * 100).sort_values(ascending=False)
missing_features = full_train_df.loc[:, missing_values > 0.0]
missing_features = missing_features.apply(mean_without_overflow_fast)
for key in full_train_df.loc[:, missing_values > 0.0].keys():
    if key == 'year_built' or key == 'floor_count':
        full_train_df[key].fillna(math.floor(missing_features[key]), inplace=True)
        full_test_df[key].fillna(math.floor(missing_features[key]), inplace=True)
    else:
        full_train_df[key].fillna(missing_features[key], inplace=True)
        full_test_df[key].fillna(missing_features[key], inplace=True)

In [54]:
full_train_df['timestamp'].dtype
# 返回数组中元素的数据类型，这个类型我都不知道是什么

In [55]:
full_train_df['timestamp']

In [56]:
full_train_df["timestamp"] = pd.to_datetime(full_train_df["timestamp"])
full_test_df["timestamp"] = pd.to_datetime(full_test_df["timestamp"])
# 表示我实在没看出来这转换完有什么区别，这个函数就是将时间转换成可读的时间
full_train_df["timestamp"]

In [57]:
def transform(df):
    df['hour'] = np.uint8(df['timestamp'].dt.hour)
    df['day'] = np.uint8(df['timestamp'].dt.day)
    df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
    df['month'] = np.uint8(df['timestamp'].dt.month)
    df['year'] = np.uint8(df['timestamp'].dt.year-1900)
    
    df['square_feet'] = np.log(df['square_feet'])
    
    return df
full_train_df = transform(full_train_df)
full_test_df = transform(full_test_df)
dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
full_train_df['is_holiday'] = (full_train_df['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
full_test_df['is_holiday'] = (full_test_df['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

In [58]:
# Assuming 5 days a week for all the given buildings
full_train_df.loc[(full_train_df['weekday'] == 5) | (full_train_df['weekday'] == 6) , 'is_holiday'] = 1
full_test_df.loc[(full_test_df['weekday']) == 5 | (full_test_df['weekday'] == 6) , 'is_holiday'] = 1

In [59]:
# 这个数据在5月20日之前看起来都是可疑的。所以把这些给删了
full_train_df = full_train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [68]:
full_test_df = full_test_df.drop(['timestamp'], axis=1)
full_train_df = full_train_df.drop(['timestamp'], axis=1)
print (f'Shape of training dataset: {full_train_df.shape}')
print (f'Shape of testing dataset: {full_test_df.shape}')

In [61]:
## Reducing memory
full_train_df = reduce_mem_usage(full_train_df)
full_test_df = reduce_mem_usage(full_test_df)
gc.collect()

In [62]:
# 这个就是风向进行转换了
def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=[i for i in range(0,16)]
    return arr[(val % 16)]
beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

for item in beaufort:
    full_train_df.loc[(full_train_df['wind_speed']>=item[1]) & (full_train_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [63]:
categoricals = ['site_id', 'building_id', 'primary_use', 'hour', 'weekday', 'meter',  'wind_direction', 'is_holiday']
drop_cols = ['sea_level_pressure', 'wind_speed']
numericals = ['square_feet', 'year_built', 'air_temperature', 'cloud_coverage',
              'dew_temperature', 'precip_depth_1_hr', 'floor_count', 'beaufort_scale']

feat_cols = categoricals + numericals

In [64]:
full_train_df = reduce_mem_usage(full_train_df)
gc.collect()

In [65]:
target = full_train_df["meter_reading"]
del full_train_df["meter_reading"]
full_train_df.drop(drop_cols, axis=1)
gc.collect()

In [67]:
# Save the testing dataset to freeup the RAM. We'll read after training
# 感觉这个老外很节省内存啊
full_test_df.to_pickle('/home/aistudio/work/full_test_df.pkl')
del full_test_df
gc.collect()

In [ ]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.4,
            'subsample_freq': 1,
            'learning_rate': 0.25,
            'num_leaves': 40,
            'feature_fraction': 0.75,
            'lambda_l1': 1,
            'lambda_l2': 1
            }

folds = 2
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

models = []
for train_index, val_index in kf.split(full_train_df, full_train_df['building_id']):
    train_X = full_train_df[feat_cols].iloc[train_index]
    val_X = full_train_df[feat_cols].iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]
    lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
    lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    models.append(gbm)
del full_train_df, train_X, val_X, lgb_train, lgb_eval, train_y, val_y, target
gc.collect()

In [ ]:
full_test_df = pd.read_pickle('/home/aistudio/work/full_test_df.pkl')
full_test_df['wind_direction'] = full_test_df['wind_direction'].apply(degToCompass)
for item in beaufort:
    full_test_df.loc[(full_test_df['wind_speed']>=item[1]) & (full_test_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [ ]:
full_test_df = full_test_df[feat_cols]

In [ ]:
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(full_test_df.shape[0]/50000)))):
    res.append(np.expm1(sum([model.predict(full_test_df.iloc[i:i+step_size]) for model in models])/folds))
    i+=step_size
res = np.concatenate(res)

In [ ]:
submission = pd.read_csv('data/data15070/sample_s.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('/home/aistudio/work/submission_fe_lgbm.csv', index=False)